In [1]:
import pandas as pd
import numpy as np
import os
import pickle 
from collections import namedtuple


In [4]:
data_dir = './Discofuse/'
wiki_dir = 'wiki/'
sports_dir = 'sports/'
file_names = ['train.tsv', 'dev.tsv', 'test.tsv']


In [5]:
wiki_train = pd.read_csv(data_dir+wiki_dir+file_names[0], sep='\t')

In [6]:
wiki_train.sample(10)

,coherent_first_sentence,coherent_second_sentence,incoherent_first_sentence,incoherent_second_sentence,discourse_type,connective_string,has_coref_type_pronoun,has_coref_type_nominal
1441403,The club managed to remain in the top division...,NaN,The club managed to remain in the top division...,The club following relegation in 2001 and inab...,SINGLE_VP_COORD,", but",0.0,0.0
2263047,Since appeasement only appeared to encourage t...,NaN,Since appeasement only appeared to encourage t...,The Japanese crew were joined by their compatr...,SINGLE_RELATIVE,NaN,0.0,0.0
2540858,The most damaging hurricane was both Hurricane...,"However , when adjusted for inflation , Katrin...",The most damaging hurricane was both Hurricane...,"When adjusted for inflation , Hurricane Katrin...",PAIR_CONN_ANAPHORA,"however ,",0.0,1.0
4427285,One copy of this tape is given to Muktijuddho ...,NaN,One copy of this tape is given to Muktijuddho ...,The original is retained by the brother of the...,SINGLE_S_COORD,", but",0.0,0.0
1976802,"This vertical palsy is supranuclear , so doll ...",NaN,This vertical palsy is supranuclear .,Doll 's head maneuver should elevate the eyes ...,SINGLE_S_COORD,", so",0.0,0.0
326256,Kila Haoda ( born 21 September 1949 ) is a Pap...,He was the Governor of Central Province from 2...,Kila Haoda ( born 21 September 1949 ) is a Pap...,Kila Haoda was the politician of Central Provi...,PAIR_ANAPHORA,NaN,1.0,1.0
3920354,"2 November : Mawlawi Abdul Manan , an importan...",NaN,2 November : Mawlawi Abdul Manan is killed by ...,Mawlawi Abdul Manan is an important Taliban fi...,SINGLE_APPOSITION,NaN,0.0,0.0
1126134,"Although as a weekend station , LWT tended to ...",NaN,As a weekend station .,LWT tended to concentrate on lighter material ...,SINGLE_CONN_START,although,0.0,0.0
3990191,"In Moldavia and Wallachia , Voievod meant the ...",NaN,"In Moldavia and Wallachia , Voievod meant the ...",Lord or master .,SINGLE_CONN_INNER,", meaning",0.0,0.0
2069856,In 2003 London Skolars were entered in League ...,NaN,In 2003 London Skolars were entered in League ...,Toulouse Olympique were entered into the Champ...,SINGLE_CONN_INNER,whereas,0.0,0.0


In [9]:
DiscoFuseExample = namedtuple('DiscoFuseExample', 'infused fused phenomanon connective')
phenomena_dict, connective_dict, phenomena_connective_dict = {}, {}, {}
wiki_raw_train_list = []

for samp in wiki_train.values:
    coherent_1, coherent_2, incoherent_1, incoherent_2, phenomenon, connective, _, _ = samp
    if not isinstance(coherent_2, str):
        fused = coherent_1
    else: 
        fused = coherent_1 + " " + coherent_2
        
    infused = incoherent_1 + " " + incoherent_2
    
    if phenomenon not in phenomena_dict:
        phenomena_dict[phenomenon] = fused
        
    if connective not in connective_dict:
        connective_dict[connective] = fused
        
    if (phenomenon, connective) not in phenomena_connective_dict:
        phenomena_connective_dict[(phenomenon, connective)] = fused
        
    wiki_raw_train_list.append(DiscoFuseExample(infused, fused, phenomenon, connective))
    


In [10]:
print(wiki_raw_train_list[0].infused)
print(wiki_raw_train_list[0].fused)


Guy Leech is still a celebrity today . Guy Leech regularly appears on television as an advocate for health and fitness .
Guy Leech is still a celebrity today and regularly appears on television as an advocate for health and fitness .


In [11]:
final_phen_conn_dict = {}

for elem in phenomena_connective_dict:
    if isinstance(elem[1], str):
        print(str(elem) + " - " + phenomena_connective_dict[elem])
        final_phen_conn_dict[elem] = phenomena_connective_dict[elem]

('SINGLE_VP_COORD', 'and') - Guy Leech is still a celebrity today and regularly appears on television as an advocate for health and fitness .
('SINGLE_CONN_INNER', ', although') - Injury , bad luck and inconsistency prevented him from ever winning the series , although he was certainly good enough .
('PAIR_CONN', 'hence ,') - Under officers are cadets who have received the necessary training and have the desire to become NZCF officers . Hence , under officers are treated as understudy officers .
('SINGLE_S_COORD', ', but') - He also stated that he contacted Strauss during the writing of the article , but Strauss only acknowledged talking to Coe 's manager , who would only comment off the record .
('SINGLE_VP_COORD', 'or') - The facility was never fully completed or used , and is currently open to visitors as a tourist attraction .
('PAIR_CONN_ANAPHORA', 'instead ,') - Lisa is transferred to Walter 's lab , where he gives her special drugs to extract Rusk 's thoughts from her mind . Ins

In [13]:
print(len(final_phen_conn_dict))

for elem in final_phen_conn_dict:
    print(elem)

150
('SINGLE_VP_COORD', 'and')
('SINGLE_CONN_INNER', ', although')
('PAIR_CONN', 'hence ,')
('SINGLE_S_COORD', ', but')
('SINGLE_VP_COORD', 'or')
('PAIR_CONN_ANAPHORA', 'instead ,')
('SINGLE_S_COORD_ANAPHORA', ', but')
('SINGLE_CONN_INNER', 'unless')
('SINGLE_VP_COORD', ', but')
('SINGLE_S_COORD', 'or')
('SINGLE_CONN_INNER', ', because')
('SINGLE_CONN_INNER', 'so that')
('SINGLE_CONN_INNER', 'although')
('SINGLE_CONN_INNER_ANAPHORA', 'so that')
('PAIR_CONN', 'for example ,')
('PAIR_CONN', 'however ,')
('PAIR_CONN', 'thus ,')
('SINGLE_CONN_INNER_ANAPHORA', ', because')
('SINGLE_CONN_INNER', ', while')
('SINGLE_S_COORD_ANAPHORA', ', so')
('SINGLE_VP_COORD', ', and')
('PAIR_CONN', 'therefore')
('SINGLE_S_COORD_ANAPHORA', 'but')
('SINGLE_CONN_INNER', 'because')
('SINGLE_CONN_START', 'although')
('PAIR_CONN_ANAPHORA', 'as a result ,')
('PAIR_CONN', 'additionally ,')
('SINGLE_CONN_INNER', 'whereas')
('SINGLE_CONN_INNER_ANAPHORA', ', while')
('SINGLE_S_COORD', 'but')
('PAIR_CONN', 'for instan

In [14]:
for elem in connective_dict:
    print(elem)

and
, although
nan
hence ,
, but
or
instead ,
unless
, because
so that
although
for example ,
however ,
thus ,
, while
, so
, and
therefore
but
because
as a result ,
additionally ,
whereas
for instance ,
therefore ,
since
finally ,
furthermore ,
on the other hand ,
, or
meaning
in the end ,
, so that
consequently ,
additionally
and although
eventually ,
in fact ,
however
in particular ,
moreover ,
hence
nevertheless
meanwhile ,
similarly ,
, meaning
, yet
then ,
but ,
for instance
for example
nevertheless ,
, nor
nor
and ,
yet ,
moreover
consequently
although ,
yet
whereas ,
or ,
furthermore
on the other hand
so
meaning ,


In [28]:
sophistication_dict = {}
# Effect
sophistication_dict['as a result ,'] = (0, 0) 
sophistication_dict['hence'] = (0, 0) 
sophistication_dict['hence ,'] = (0, 0)  
sophistication_dict['consequently'] = (1, 0) 
sophistication_dict['consequently ,'] = (1, 0) 
sophistication_dict['thus ,'] = (0, 0) 
sophistication_dict['therefore'] = (1, 0) 
sophistication_dict['therefore ,'] = (1, 0) 

# Adding
sophistication_dict['furthermore'] = (1, 1) 
sophistication_dict['furthermore ,'] = (1, 1)
sophistication_dict['moreover'] = (1, 1)
sophistication_dict['moreover ,'] = (1, 1) 
sophistication_dict['plus ,'] = (1, 1)
sophistication_dict['additionally'] = (1, 1)
sophistication_dict['additionally ,'] = (1, 1)
sophistication_dict[', and'] = (0, 1)
sophistication_dict['and'] = (0, 1)
sophistication_dict['and ,'] = (0, 1)

# Sequencing
sophistication_dict['finally ,'] = (1, 2) 
sophistication_dict['eventually ,'] = (1, 2) 
sophistication_dict['in the end ,'] = (0, 2)

# Contrasting
sophistication_dict['whereas'] = (1, 3)
sophistication_dict['whereas ,'] = (1, 3)
sophistication_dict['nevertheless'] = (1, 3)
sophistication_dict['nevertheless ,'] = (1, 3)
sophistication_dict['on the other hand'] = (0, 3)
sophistication_dict['on the other hand ,'] = (0, 3)

# Qualifying
sophistication_dict['however'] = (1, 4)
sophistication_dict['however ,'] = (1, 4)
sophistication_dict[', although'] = (1, 4)
sophistication_dict['although'] = (1, 4)
sophistication_dict['although ,'] = (1, 4)
sophistication_dict['still ,'] = (1, 4)
sophistication_dict[', yet'] = (0, 4)
sophistication_dict['yet'] = (0, 4)
sophistication_dict['yet ,'] = (0, 4)
sophistication_dict[', but'] = (0, 4)
sophistication_dict['but'] = (0, 4)

# Illustrating
sophistication_dict['for instance'] = (1, 5)
sophistication_dict['for instance ,'] = (1, 5)
sophistication_dict['for example'] = (0, 5)
sophistication_dict['for example ,'] = (0, 5)


In [39]:
sem2id = {'Effect': 0, 'Adding':1, 'Sequencing':2, 'Contrasting': 3, 'Qualifying':4, 'Illustrating': 5}
id2sem = {0: 'Effect', 1: 'Adding', 2: 'Sequencing', 3: 'Contrasting', 4: 'Qualifying', 5: 'Illustrating'}

In [31]:
DiscoFuseExampleCI = namedtuple('DiscoFuseExampleCI', 'infused fused phenomanon connective sophisticated semantic')
wiki_train_list = []
semantic_sophist_count_wiki = np.zeros([2, 6])

for samp in wiki_train.values:
    coherent_1, coherent_2, incoherent_1, incoherent_2, phenomenon, connective, _, _ = samp
    if not isinstance(coherent_2, str):
        fused = coherent_1
    else: 
        fused = coherent_1 + " " + coherent_2
        
    infused = incoherent_1 + " " + incoherent_2
    
    if connective in sophistication_dict:
        (cur_soph, cur_sem) = sophistication_dict[connective]
        wiki_train_list.append(DiscoFuseExampleCI(infused, fused, phenomenon, connective, cur_soph, cur_sem))
        semantic_sophist_count_wiki[cur_soph][cur_sem] += 1
     


4490804
2110554


In [36]:
print(len(wiki_raw_train_list))
print(len(wiki_train_list))

4490804
2110554


In [33]:
sports_train = pd.read_csv(data_dir+sports_dir+file_names[0], sep='\t')

In [34]:
sports_train_list = []
semantic_sophist_count_sports = np.zeros([2, 6])

for samp in sports_train.values:
    coherent_1, coherent_2, incoherent_1, incoherent_2, phenomenon, connective, _, _ = samp
    if not isinstance(coherent_2, str):
        fused = coherent_1
    else: 
        fused = coherent_1 + " " + coherent_2
        
    infused = incoherent_1 + " " + incoherent_2
    
    if connective in sophistication_dict:
        (cur_soph, cur_sem) = sophistication_dict[connective]
        sports_train_list.append(DiscoFuseExampleCI(infused, fused, phenomenon, connective, cur_soph, cur_sem))
        semantic_sophist_count_sports[cur_soph][cur_sem] += 1

In [42]:
print(len(sports_train))
print(len(sports_train_list))
print(len(semantic_sophist_count_wiki[0]))

11838624
4176676
6


In [46]:
print("Wiki:")
for i in range(len(semantic_sophist_count_wiki[0])):
    non_soph_per =  semantic_sophist_count_wiki[0][i] / (semantic_sophist_count_wiki[0][i] + semantic_sophist_count_wiki[1][i])
    soph_per =  semantic_sophist_count_wiki[1][i] / (semantic_sophist_count_wiki[0][i] + semantic_sophist_count_wiki[1][i])
    print(id2sem[i] + " - [" + str(non_soph_per) + ", " + str(soph_per) + "]")

print()
print("Sports:")
for i in range(len(semantic_sophist_count_sports[0])):
    non_soph_per =  semantic_sophist_count_sports[0][i] / (semantic_sophist_count_sports[0][i] + semantic_sophist_count_sports[1][i])
    soph_per =  semantic_sophist_count_sports[1][i] / (semantic_sophist_count_sports[0][i] + semantic_sophist_count_sports[1][i])
    print(id2sem[i] + " - [" + str(non_soph_per) + ", " + str(soph_per) + "]")


Wiki:
Effect - [0.675322643723113, 0.324677356276887]
Adding - [0.9130033900072166, 0.0869966099927834]
Sequencing - [0.19144421208020668, 0.8085557879197933]
Contrasting - [0.11039912690988463, 0.8896008730901154]
Qualifying - [0.3961094138913261, 0.603890586108674]
Illustrating - [0.8361754951885613, 0.16382450481143873]

Sports:
Effect - [0.6097138298750241, 0.3902861701249759]
Adding - [0.9616577871776888, 0.03834221282231125]
Sequencing - [0.0, 1.0]
Contrasting - [0.21164586593387022, 0.7883541340661298]
Qualifying - [0.5566844169253422, 0.4433155830746578]
Illustrating - [1.0, 0.0]


In [47]:
train_list = wiki_train_list + sports_train_list
with open("./wiki_sports_train.pkl", 'wb') as fp:
    pickle.dump(train_list, fp)

In [49]:
wiki_dev = pd.read_csv(data_dir+wiki_dir+file_names[1], sep='\t')
sports_dev = pd.read_csv(data_dir+sports_dir+file_names[1], sep='\t')

In [54]:
wiki_dev_list = []
dev_semantic_sophist_count_wiki = np.zeros([2, 6])

for samp in wiki_dev.values:
    coherent_1, coherent_2, incoherent_1, incoherent_2, phenomenon, connective, _, _ = samp
    if not isinstance(coherent_2, str):
        fused = coherent_1
    else: 
        fused = coherent_1 + " " + coherent_2
        
    infused = incoherent_1 + " " + incoherent_2
    
    if connective in sophistication_dict:
        (cur_soph, cur_sem) = sophistication_dict[connective]
        wiki_dev_list.append(DiscoFuseExampleCI(infused, fused, phenomenon, connective, cur_soph, cur_sem))
        dev_semantic_sophist_count_wiki[cur_soph][cur_sem] += 1

In [55]:
sports_dev_list = []
dev_semantic_sophist_count_sports = np.zeros([2, 6])

for samp in sports_dev.values:
    coherent_1, coherent_2, incoherent_1, incoherent_2, phenomenon, connective, _, _ = samp
    if not isinstance(coherent_2, str):
        fused = coherent_1
    else: 
        fused = coherent_1 + " " + coherent_2
        
    infused = incoherent_1 + " " + incoherent_2
    
    if connective in sophistication_dict:
        (cur_soph, cur_sem) = sophistication_dict[connective]
        sports_dev_list.append(DiscoFuseExampleCI(infused, fused, phenomenon, connective, cur_soph, cur_sem))
        dev_semantic_sophist_count_sports[cur_soph][cur_sem] += 1

In [56]:
print("Wiki:")
for i in range(len(dev_semantic_sophist_count_wiki[0])):
    non_soph_per =  dev_semantic_sophist_count_wiki[0][i] / (dev_semantic_sophist_count_wiki[0][i] + dev_semantic_sophist_count_wiki[1][i])
    soph_per =  dev_semantic_sophist_count_wiki[1][i] / (dev_semantic_sophist_count_wiki[0][i] + dev_semantic_sophist_count_wiki[1][i])
    print(id2sem[i] + " - [" + str(non_soph_per) + ", " + str(soph_per) + "]")

print()
print("Sports:")
for i in range(len(dev_semantic_sophist_count_sports[0])):
    non_soph_per =  dev_semantic_sophist_count_sports[0][i] / (dev_semantic_sophist_count_sports[0][i] + dev_semantic_sophist_count_sports[1][i])
    soph_per =  dev_semantic_sophist_count_sports[1][i] / (dev_semantic_sophist_count_sports[0][i] + dev_semantic_sophist_count_sports[1][i])
    print(id2sem[i] + " - [" + str(non_soph_per) + ", " + str(soph_per) + "]")

Wiki:
Effect - [0.640625, 0.359375]
Adding - [0.9186697398748765, 0.08133026012512348]
Sequencing - [0.18862275449101795, 0.811377245508982]
Contrasting - [0.11641791044776119, 0.8835820895522388]
Qualifying - [0.3952015203103967, 0.6047984796896033]
Illustrating - [0.8174204355108877, 0.18257956448911222]

Sports:
Effect - [0.6114381833473507, 0.3885618166526493]
Adding - [0.9582856758393326, 0.04171432416066743]
Sequencing - [0.0, 1.0]
Contrasting - [0.21834625322997417, 0.7816537467700259]
Qualifying - [0.5510294711344368, 0.44897052886556316]
Illustrating - [1.0, 0.0]


In [57]:
with open("./wiki_dev.pkl", 'wb') as fp:
    pickle.dump(wiki_dev_list, fp)
    
with open("./sports_dev.pkl", 'wb') as fp:
    pickle.dump(sports_dev_list, fp)

[DiscoFuseExampleCI(infused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 . Falsely dated to 1810 .', fused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 , although falsely dated to 1810 .', phenomanon='SINGLE_CONN_INNER', connective=', although', sophisticated=1, semantic=4)]
[DiscoFuseExampleCI(infused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 . Falsely dated to 1810 .', fused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 , although falsely dated to 1810 .', phenomanon='SINGLE_CONN_INNER', connective=', although', sophisticated=1, semantic=4)]


[DiscoFuseExampleCI(infused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 . Falsely dated to 1810 .', fused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 , although falsely dated to 1810 .', phenomanon='SINGLE_CONN_INNER', connective=', although', sophisticated=1, semantic=4), DiscoFuseExampleCI(infused='Henderson places it in The Library Illustrative of Social Progress published around 1872 ( falsely dated 1777 ) . It is not in the list of Henry Spencer Ashbee .', fused='Henderson places it in The Library Illustrative of Social Progress published around 1872 ( falsely dated 1777 ) but it is not in the list of Henry Spencer Ashbee .', phenomanon='SINGLE_S_COORD', connective='but', sophisticated=0, semantic=4)]
[DiscoFuseExampleCI(infused='The Rodiad is a pornographic poem on the subject of flagellation published by John Camden Hotten in 1871 . Falsely dated to 1810 .'

In [3]:
DiscoFuseExampleCI = namedtuple('DiscoFuseExampleCI', 'infused fused phenomanon connective sophisticated semantic')

with open("./wiki_sports_train.pkl", 'rb') as fp:
    t_list = pickle.load(fp)